In [ ]:
import os, re, random, time, ssl, smtplib
from email.message import EmailMessage
from datetime import datetime, date, timezone
from zoneinfo import ZoneInfo
import mysql.connector
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from datetime import timedeltasffffv


In [2]:
# -------------------- ENV & CONFIG --------------------
load_dotenv(find_dotenv())

# Toggle: start the minute-by-minute sender after inserting (False = just insert)
RUN_SCHEDULER_AFTER_INSERT = False

APP_TZ   = ZoneInfo(os.getenv("APP_TZ", "Asia/Kolkata"))

DB = dict(
    host=os.getenv("DB_HOST", "127.0.0.1"),
    port=int(os.getenv("DB_PORT", "3307")),   # change to 3307 if your server uses that
    user=os.getenv("DB_USER", "reminder_app"),
    password=os.getenv("DB_PASS", "App#Pass123!"),
    database=os.getenv("DB_NAME", "reminder_app"),
)

SMTP_HOST = os.getenv("SMTP_HOST", "smtp.gmail.com")
SMTP_PORT = int(os.getenv("SMTP_PORT", "587"))   # 587 (STARTTLS) or 465 (SSL)
SMTP_USER = os.getenv("SMTP_USER")               # your Gmail address
SMTP_PASS = os.getenv("SMTP_PASS")               # 16-char Gmail App Password
FROM_EMAIL = os.getenv("FROM_EMAIL", SMTP_USER)

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_KEY)

In [3]:
TIME_RE = re.compile(r"^(?:[01]\d|2[0-3]):[0-5]\d$")  # strict HH:MM (00-23:00-59)

def get_hhmm(s: str) -> str:
    s = s.strip()
    if not TIME_RE.match(s):
        raise ValueError("Time must be HH:MM (00–23:00–59)")
    return s

# --- Helpers ---
def fallback_quote(topic: str) -> str:
    t = re.sub(r"[^A-Za-z0-9 ']", "", topic).strip().capitalize() or "Life"
    bank = [
        "{t} grows when we give more than we guard.",
        "In {t}, the quiet choices shape the loudest days.",
        "{t} is courage choosing kindness again and again.",
        "When {t} leads, fear forgets its lines.",
        "{t} makes ordinary moments feel like destiny.",
    ]
    return random.choice(bank).format(t=t)


In [4]:
def generate_quote(topic: str) -> str:
    """Generate one-sentence quote (<=120 chars) for the topic. Falls back locally if API/quota fails."""
    prompt = (
        f"Write ONE original, memorable quote about '{topic}'. "
        "Exactly one sentence, ≤120 characters, no emojis/hashtags/author, "
        "no surrounding quotation marks. Return only the sentence."
    )
    try:
        # Request the AI API for a quote
        r = client.responses.create(model="gpt-4o-mini", input=prompt, max_output_tokens=120)
        text = (r.output_text or "").strip()

        # Check if the response is valid (non-empty and meaningful)
        if not text or len(text) < 10:  # Can adjust this condition based on expected behavior
            print(f"Generated text is invalid or too short: {text}")
            raise ValueError("API response is invalid.")
        
    except Exception as e:
        print("LLM unavailable or invalid response → fallback used:", str(e)[:120])
        text = fallback_quote(topic)

    # Clean up any surrounding quotes or invalid characters
    text = text.strip().strip('"\''"“”'")

    # Ensure the quote length is <= 120 characters
    return text if len(text) <= 120 else (text[:119] + "…")

In [5]:
def send_email(to: str, subject: str, body: str):
    msg = EmailMessage()
    msg["From"] = FROM_EMAIL
    msg["To"] = to
    msg["Subject"] = subject
    msg.set_content(body)

    if SMTP_PORT == 465:
        with smtplib.SMTP_SSL(SMTP_HOST, 465, context=ssl.create_default_context()) as s:
            s.login(SMTP_USER, SMTP_PASS)
            s.send_message(msg)
    else:
        with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as s:
            s.starttls(context=ssl.create_default_context())
            s.login(SMTP_USER, SMTP_PASS)  # Gmail App Password
            s.send_message(msg)

In [6]:
def get_or_create_user_id(conn, name: str) -> int:
    """Return existing user's id (case-insensitive match), else insert new user."""
    cur = conn.cursor()
    try:
        cur.execute("SELECT id FROM users WHERE LOWER(name)=LOWER(%s) ORDER BY id ASC LIMIT 1", (name,))
        row = cur.fetchone()
        if row:
            return row[0]
        cur.execute("INSERT INTO users (name) VALUES (%s)", (name,))
        conn.commit()
        return cur.lastrowid
    finally:
        cur.close()

In [7]:
def insert_reminder(conn, user_id: int, subject_topic: str, time_hhmm: str, recipient_email: str) -> int:
    cur = conn.cursor()
    try:
        cur.execute(
            "INSERT INTO reminders (user_id, subject, time_hhmm, recipient_email) "
            "VALUES (%s, %s, %s, %s)",
            (user_id, subject_topic, time_hhmm, recipient_email),
        )
        conn.commit()
        return cur.lastrowid
    finally:
        cur.close()

In [8]:
def send_due_reminders_exact_minute() -> int:
    """Send only when time_hhmm == current minute. Returns #emails sent."""
    now_local   = datetime.now(APP_TZ)
    hhmm        = now_local.strftime("%H:%M")
    today_local = now_local.date()
    sent = 0

    cn = mysql.connector.connect(**DB)
    try:
        cur = cn.cursor(dictionary=True)
        cur.execute("""
            SELECT r.id, r.user_id, r.subject, r.time_hhmm, r.recipient_email, u.name
            FROM reminders r
            JOIN users u ON u.id = r.user_id
            WHERE r.time_hhmm = %s AND COALESCE(r.recipient_email,'') <> ''
        """, (hhmm,))
        due = cur.fetchall()

        print(f"[{now_local}] hhmm={hhmm} exact matches={len(due)}")

        for row in due:
            # once-per-day protection
            cur.execute(
                "SELECT 1 FROM reminder_deliveries WHERE reminder_id=%s AND delivered_on=%s LIMIT 1",
                (row["id"], today_local),
            )
            if cur.fetchone():
                continue

            topic = row["subject"]
            quote = generate_quote(topic)
            body  = f"Hi {row['name']}\n\n{quote}"
            try:
                send_email(row["recipient_email"], topic, body)
                cur.execute(
                    "INSERT INTO reminder_deliveries (reminder_id, delivered_on) VALUES (%s, %s)",
                    (row["id"], today_local),
                )
                cn.commit()
                sent += 1
                print(f"📧 sent reminder {row['id']} to {row['recipient_email']} at {hhmm}")
                
            except Exception as e:
                cn.rollback()
                print(f"❌ send failed for reminder {row['id']}: {e}")
    finally:
        cn.close()
    return sent


In [9]:
print("Enter details for a new reminder (subject is the TOPIC; quote will be emailed at send time).")
name        = input("Name: ").strip()
subject     = input("Topic (saved as subject): ").strip()
time_hhmm   = get_hhmm(input("Time (HH:MM 24h): "))
recipient   = input("Recipient email: ").strip()

conn = mysql.connector.connect(**DB)
try:
    user_id = get_or_create_user_id(conn, name)             # only creates user if new
    reminder_id = insert_reminder(conn, user_id, subject, time_hhmm, recipient)
    print(f"✅ Saved reminder #{reminder_id} for user #{user_id} (topic='{subject}', time={time_hhmm})")
finally:
    conn.close() 

Enter details for a new reminder (subject is the TOPIC; quote will be emailed at send time).
✅ Saved reminder #28 for user #4 (topic='Money', time=19:22)


In [10]:
def run_exact_minute_loop(stop_after_first_send: bool = False, max_runtime_minutes: int | None = None):
    """Minute-precise loop. If stop_after_first_send=True, exits after first successful send."""
    start = datetime.now(APP_TZ)
    print(f"Exact-minute scheduler running (APP_TZ={APP_TZ}). Ctrl+C to stop.")
    while True:
        # sleep to next :00
        now = datetime.now(APP_TZ)
        next_minute = (now.replace(second=0, microsecond=0) + timedelta(minutes=1))
        time.sleep(max(0.5, (next_minute - now).total_seconds()))
        sent = send_due_reminders_exact_minute()
        if stop_after_first_send and sent > 0:
            print("✅ First send done — stopping loop.")
            break
        if max_runtime_minutes is not None and (datetime.now(APP_TZ) - start).total_seconds() > max_runtime_minutes*60:
            print("⏱️ Max runtime reached — stopping loop.")
            break

In [11]:
run_exact_minute_loop(stop_after_first_send=True)

Exact-minute scheduler running (APP_TZ=Asia/Kolkata). Ctrl+C to stop.
[2025-09-16 19:22:00.000771+05:30] hhmm=19:22 exact matches=1
📧 sent reminder 28 to jomfolives651@gmail.com at 19:22
✅ First send done — stopping loop.
